In [1]:
from os import path
from shutil import copyfile
from fluctmatch import avg_dcd_noh

rootfolder = '/home/yizaochen/codes/dna_rna/all_systems'
na_mechfolder = '/home/yizaochen/codes/na_mechanics'
simu_folder = '/home/ytcdata/simulation'
d_single_na = {'arna+arna': 'arna', 'bdna+bdna': 'bdna'}
vmd = '/usr/local/bin/vmd'

### Step 1: Initialize

In [2]:
host = 'a_tract_21mer'
type_na = 'bdna+bdna'
n_bp = 21
begin_frame = 1
frame_num = 50000

agent = avg_dcd_noh.AvgcrddcdAgent(host, type_na, rootfolder)

/home/yizaochen/codes/dna_rna/all_systems/a_tract_21mer exists
/home/yizaochen/codes/dna_rna/all_systems/a_tract_21mer/bdna+bdna exists
/home/yizaochen/codes/dna_rna/all_systems/a_tract_21mer/bdna+bdna/input exists
/home/yizaochen/codes/dna_rna/all_systems/a_tract_21mer/bdna+bdna/input/allatoms exists
/home/yizaochen/codes/dna_rna/all_systems/a_tract_21mer/bdna+bdna/input/heavyatoms exists
/home/yizaochen/codes/dna_rna/all_systems/a_tract_21mer/bdna+bdna/charmm_inp exists
/home/yizaochen/codes/dna_rna/all_systems/a_tract_21mer/bdna+bdna/charmm_dat exists
/home/yizaochen/codes/dna_rna/all_systems/a_tract_21mer/bdna+bdna/make_crd exists


#### Copy central.xtc from simulation folder 

In [3]:
xtc_0us_5us = path.join(simu_folder, host, type_na, 'data', 'roughtrj', '1000', f'{type_na}.nopbc.fit.1to50.1000.xtc')
central_xtc = path.join(agent.aa_folder, f'{type_na}.central.xtc')
copyfile(xtc_0us_5us, central_xtc)
print(f'cp {xtc_0us_5us} {central_xtc}')

cp /home/ytcdata/simulation/tat_21mer/bdna+bdna/data/roughtrj/1000/bdna+bdna.nopbc.fit.1to50.1000.xtc /home/yizaochen/codes/dna_rna/all_systems/tat_21mer/bdna+bdna/input/allatoms/bdna+bdna.central.xtc


### Step 2: Prepare dcd and pdb

In [4]:
inish = path.join(na_mechfolder, 'shell_scripts', 'initialize_input.sh')
cmd = f'bash {inish} {rootfolder} {host} {type_na}'
print(cmd)
cmd = f'r 1-{n_bp}'
print(cmd)

bash /home/yizaochen/codes/na_mechanics/shell_scripts/initialize_input.sh /home/yizaochen/codes/dna_rna/all_systems tat_21mer bdna+bdna
r 1-21


In [5]:
# Manual Delete
aafolder = agent.aa_folder
single_na = d_single_na[type_na]
temp_pdb = path.join(aafolder, f'{single_na}1.central.pdb')
cmd = f'vim {temp_pdb}'
print(cmd)
temp_pdb = path.join(aafolder, f'{single_na}2.central.pdb')
cmd = f'vim {temp_pdb}'
print(cmd)

vim /home/yizaochen/codes/dna_rna/all_systems/tat_21mer/bdna+bdna/input/allatoms/bdna1.central.pdb
vim /home/yizaochen/codes/dna_rna/all_systems/tat_21mer/bdna+bdna/input/allatoms/bdna2.central.pdb


In [6]:
pdbcharmmsh = path.join(na_mechfolder, 'shell_scripts', 'pdb_gro2charmm.sh')
cmd = f'bash {pdbcharmmsh} {rootfolder} {host} {type_na} 1'
print(cmd)
cmd = f'bash {pdbcharmmsh} {rootfolder} {host} {type_na} 2'
print(cmd)

bash /home/yizaochen/codes/na_mechanics/shell_scripts/pdb_gro2charmm.sh /home/yizaochen/codes/dna_rna/all_systems tat_21mer bdna+bdna 1
bash /home/yizaochen/codes/na_mechanics/shell_scripts/pdb_gro2charmm.sh /home/yizaochen/codes/dna_rna/all_systems tat_21mer bdna+bdna 2


In [7]:
temp_pdb = path.join(agent.aa_folder, f'{type_na}.central.pdb')
temp_xtc = path.join(agent.aa_folder, f'{type_na}.central.xtc')
temp_dcd = path.join(agent.aa_folder, f'{type_na}.central.dcd')
cmd = f'{vmd} {temp_pdb} {temp_xtc}'
print(cmd)
cmd = f'animate write dcd {temp_dcd} beg 1 end 50001 waitfor all'
print(cmd)
cmd = f'{vmd} {temp_pdb} {temp_dcd}'
print(cmd)

/usr/local/bin/vmd /home/yizaochen/codes/dna_rna/all_systems/tat_21mer/bdna+bdna/input/allatoms/bdna+bdna.central.pdb /home/yizaochen/codes/dna_rna/all_systems/tat_21mer/bdna+bdna/input/allatoms/bdna+bdna.central.xtc
animate write dcd /home/yizaochen/codes/dna_rna/all_systems/tat_21mer/bdna+bdna/input/allatoms/bdna+bdna.central.dcd beg 1 end 50001 waitfor all
/usr/local/bin/vmd /home/yizaochen/codes/dna_rna/all_systems/tat_21mer/bdna+bdna/input/allatoms/bdna+bdna.central.pdb /home/yizaochen/codes/dna_rna/all_systems/tat_21mer/bdna+bdna/input/allatoms/bdna+bdna.central.dcd


### Step 3: Make CRD (split two strands, then combine) 

In [8]:
agent.make_crd_input(amber=True, firstter='amber_5ter', lastter='amber_3ter')

In [12]:
agent.make_crd()

charmm< /home/yizaochen/codes/dna_rna/all_systems/tat_21mer/bdna+bdna/make_crd/bdna1.inp > /home/yizaochen/codes/dna_rna/all_systems/tat_21mer/bdna+bdna/make_crd/bdna1.dat
charmm< /home/yizaochen/codes/dna_rna/all_systems/tat_21mer/bdna+bdna/make_crd/bdna2.inp > /home/yizaochen/codes/dna_rna/all_systems/tat_21mer/bdna+bdna/make_crd/bdna2.dat
charmm< /home/yizaochen/codes/dna_rna/all_systems/tat_21mer/bdna+bdna/make_crd/bdna+bdna.inp > /home/yizaochen/codes/dna_rna/all_systems/tat_21mer/bdna+bdna/make_crd/bdna+bdna.dat


In [11]:
# Reset Resid for bdna2.1.pdb, if need
execute = False
if execute:
    offset = -21
    agent.reset_na2_pdb_resid(offset)

/home/yizaochen/codes/dna_rna/all_systems/tat_21mer/bdna+bdna/make_crd/bdna2.1.pdb /home/yizaochen/codes/dna_rna/all_systems/tat_21mer/bdna+bdna/make_crd/bdna2.1.backup.pdb
Write PDB: /home/yizaochen/codes/dna_rna/all_systems/tat_21mer/bdna+bdna/make_crd/bdna2.1.pdb
Reset /home/yizaochen/codes/dna_rna/all_systems/tat_21mer/bdna+bdna/make_crd/bdna2.1.pdb resid by offset -21!
Check by...
vim /home/yizaochen/codes/dna_rna/all_systems/tat_21mer/bdna+bdna/make_crd/bdna2.1.pdb


### Step 4: Make CRD and DCD without hydrogen atoms

In [13]:
agent.make_no_h_crd_input(amber=True, firstter='amber_5ter', lastter='amber_3ter')

In [14]:
agent.make_no_h_crd()

charmm< /home/yizaochen/codes/dna_rna/all_systems/tat_21mer/bdna+bdna/charmm_inp/write_no_h_crd.inp > /home/yizaochen/codes/dna_rna/all_systems/tat_21mer/bdna+bdna/charmm_dat/write_no_h_crd.dat


In [15]:
agent.make_no_h_dcd_input(amber=True, begin=begin_frame, frame_num=frame_num, firstter='amber_5ter', lastter='amber_3ter')

In [16]:
agent.make_no_h_dcd()

charmm< /home/yizaochen/codes/dna_rna/all_systems/tat_21mer/bdna+bdna/charmm_inp/write_no_h_dcd.inp > /home/yizaochen/codes/dna_rna/all_systems/tat_21mer/bdna+bdna/charmm_dat/write_no_h_dcd.dat


### Step 5: Make Average CRD and fitting no-H dcd to average crd

In [3]:
agent.make_avg_crd_input(amber=True, firstter='amber_5ter', lastter='amber_3ter')

In [4]:
agent.make_avg_crd()

charmm< /home/yizaochen/codes/dna_rna/all_systems/a_tract_21mer/bdna+bdna/charmm_inp/write_no_h_avg_crd.inp > /home/yizaochen/codes/dna_rna/all_systems/a_tract_21mer/bdna+bdna/charmm_dat/write_no_h_avg_crd.dat


In [5]:
agent.fit_dcd_to_avg_input(amber=True, begin=begin_frame, frame_num=frame_num, firstter='amber_5ter', lastter='amber_3ter')

In [6]:
agent.fit_dcd_to_avg()

charmm< /home/yizaochen/codes/dna_rna/all_systems/a_tract_21mer/bdna+bdna/charmm_inp/fit_dcd_to_avg.inp > /home/yizaochen/codes/dna_rna/all_systems/a_tract_21mer/bdna+bdna/charmm_dat/fit_dcd_to_avg.dat


### Step 6: Check By VMD

In [21]:
crd = path.join(agent.heavy_folder, f'{type_na}.nohydrogen.avg.crd')
dcd = path.join(agent.heavy_folder, f'{type_na}.nohydrogen.fitavg.dcd')
print(f'vmd -cor {crd} {dcd}')

vmd -cor /home/yizaochen/codes/dna_rna/all_systems/tat_21mer/bdna+bdna/input/heavyatoms/bdna+bdna.nohydrogen.avg.crd /home/yizaochen/codes/dna_rna/all_systems/tat_21mer/bdna+bdna/input/heavyatoms/bdna+bdna.nohydrogen.fitavg.dcd


### Additional Part: Copy requried files to allsystem folder

In [58]:
# Copy From simulation folder
simu_folder = '/home/yizaochen/simulation'
simu_datafolder = path.join(simu_folder, host, type_na, 'data')
inputfolder = path.join(rootfolder, host, type_na, 'input', 'allatoms')

old_f = path.join(simu_datafolder, 'gro', f'{type_na}.npt4.fit.gro')
new_f = path.join(inputfolder, f'{type_na}.npt4.all.gro')
copyfile(old_f, new_f)
print(f'cp {old_f} {new_f}')

old_f = path.join(simu_datafolder, 'roughtrj', '1000', f'{type_na}.nopbc.fit.1to10.1000.xtc')
new_f = path.join(inputfolder, f'{type_na}.all.xtc')
copyfile(old_f, new_f)
print(f'cp {old_f} {new_f}')

old_f = path.join(simu_folder, host, type_na, f'{type_na}.gro')
new_f = path.join(inputfolder, f'{type_na}.perfect.gro')
copyfile(old_f, new_f)
print(f'cp {old_f} {new_f}')

cp /home/yizaochen/simulation/gcgc_21mer/bdna+bdna/data/gro/bdna+bdna.npt4.fit.gro /home/yizaochen/codes/dna_rna/all_systems/gcgc_21mer/bdna+bdna/input/allatoms/bdna+bdna.npt4.all.gro
cp /home/yizaochen/simulation/gcgc_21mer/bdna+bdna/data/roughtrj/1000/bdna+bdna.nopbc.fit.1to10.1000.xtc /home/yizaochen/codes/dna_rna/all_systems/gcgc_21mer/bdna+bdna/input/allatoms/bdna+bdna.all.xtc
cp /home/yizaochen/simulation/gcgc_21mer/bdna+bdna/bdna+bdna.gro /home/yizaochen/codes/dna_rna/all_systems/gcgc_21mer/bdna+bdna/input/allatoms/bdna+bdna.perfect.gro


### Reload Function

In [15]:
from imp import reload
reload(avg_dcd_noh)

<module 'avg_dcd_noh' from '/home/yizaochen/codes/na_mechanics/avg_dcd_noh.py'>